# Combine metrics from all datasets and prepare one csv file for 1x1 degree for India from 01-01-2019 to 30-09-2021

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.options.mode.use_inf_as_na = True
import numpy as np
import os
import math
from scipy.stats import pearsonr as corr
from matplotlib import pyplot as plt
#import statsmodels.api as sm
import statsmodels.formula.api as sm
import datetime as dt

In [ ]:
AIRS_PATH="airs/data/" #path for air dataset
SNOWCOVER_PATH="snowcover/data/" #path for snowcover dataset
VEG_PATH="vegetation/data/" #path for vegetation dataset
NTL_PATH="ntl/data/" #path for night time light data
OMINO2_PATH="omino2/data/" # path for nitrogen dioxide data
COVID_PATH="covidoxford/data/" #path for covid oxford data
CITIES_PATH="cities/data/" # path for cities data
OUTPUT_PATH="data/" #path to store the data

In [ ]:
airs_df=pd.read_csv(AIRS_PATH+"indian_cities_1degx1deg_airs.csv")
airs_df=airs_df[(airs_df["mean"]>-9999)&(~airs_df["mean"].isna())]

In [ ]:
print(airs_df[~airs_df["metric"].str.contains("#")]["metric"].unique().tolist())

['SurfPres_Forecast_TqJ_A', 'SurfSkinTemp_TqJ_A', 'SurfAirTemp_TqJ_A', 'TropPres_TqJ_A', 'TropTemp_TqJ_A', 'TotH2OVap_TqJ_A', 'H2O_MMR_Surf_TqJ_A', 'RelHumSurf_TqJ_A', 'RelHumSurf_liquid_TqJ_A', 'TropHeight_TqJ_A', 'CloudFrc_TqJ_A', 'CloudTopPres_TqJ_A', 'CloudTopTemp_TqJ_A', 'TotO3_TqJ_A', 'OLR_TqJ_A', 'ClrOLR_TqJ_A', 'SurfPres_Forecast_TqJ_D', 'SurfSkinTemp_TqJ_D', 'SurfAirTemp_TqJ_D', 'TropPres_TqJ_D', 'TropTemp_TqJ_D', 'TotH2OVap_TqJ_D', 'H2O_MMR_Surf_TqJ_D', 'RelHumSurf_TqJ_D', 'RelHumSurf_liquid_TqJ_D', 'TropHeight_TqJ_D', 'CloudFrc_TqJ_D', 'CloudTopPres_TqJ_D', 'CloudTopTemp_TqJ_D', 'TotO3_TqJ_D', 'OLR_TqJ_D', 'ClrOLR_TqJ_D']


**Following day time metrics are taken**

In [ ]:
airs_metrics=['SurfSkinTemp_TqJ_A', 'SurfAirTemp_TqJ_A', 'TropPres_TqJ_A',
              'TropTemp_TqJ_A', 'TotH2OVap_TqJ_A', 'H2O_MMR_Surf_TqJ_A', 'RelHumSurf_TqJ_A',
              'RelHumSurf_liquid_TqJ_A', 'TropHeight_TqJ_A', 'CloudFrc_TqJ_A', 'CloudTopPres_TqJ_A',
              'CloudTopTemp_TqJ_A', 'TotO3_TqJ_A', 'OLR_TqJ_A']

In [ ]:
airs_df=airs_df[airs_df["metric"].isin(airs_metrics)][["location","week","metric","mean"]]

**EVI and NDVI from vegetation data**

In [ ]:
veg_df=pd.read_csv(VEG_PATH+"indian_cities_1degx1deg_vegetation.csv")
veg_df["metric"].unique()

array(['CMG 0.05 Deg 16 days NDVI', 'CMG 0.05 Deg 16 days EVI'],
      dtype=object)

In [ ]:
metrics=['CMG 0.05 Deg 16 days NDVI', 'CMG 0.05 Deg 16 days EVI']
veg_df=veg_df[(veg_df["metric"].isin(metrics))&(veg_df["mean"]>-3000)&
              (~veg_df["mean"].isna())][["location","week","metric","mean"]]

# 4 weeks a month
Vegetation data is 16-day data. Rest all data are available at 8-day or mapped to 8-day from daily data. The process for those are available in their respective files in data preparation notebooks folder. <br><br>

So, to map vegetation data to 4 weeks a month data is copied.<br> For week1 and week 2, vegetation data week1 is only used and for week3 and week4, week2 of vegetation is used. Hence both are used twice and made to 4 weeks a month.

In [ ]:
veg_df['week'] = veg_df['week'].str.replace('week2','week3')
veg_copy_df=veg_df.copy(deep=True)
veg_copy_df['week'] = veg_copy_df['week'].str.replace('week1','week2')
veg_copy_df['week'] = veg_copy_df['week'].str.replace('week3','week4')
veg_df=veg_df.append(veg_copy_df,ignore_index=True)

**metric snow level is used**

In [ ]:
sc_df=pd.read_csv(SNOWCOVER_PATH+"indian_cities_1degx1deg_snowcover.csv")
sc_df["metric"].unique()

array(['Eight_Day_CMG_Snow_Cover'], dtype=object)

In [ ]:
sc_df=sc_df[(sc_df["metric"]=="Eight_Day_CMG_Snow_Cover")&
            (sc_df["mean"]>=0)&(sc_df["mean"]<=100)&(~sc_df["mean"].isna())][["location","week","metric","mean"]]

**From Night Time Light Gap_Filled_DNB_BRDF-Corrected_NTL metric is used**

In [ ]:
ntl_df=pd.read_csv(NTL_PATH+"indian_cities_1degx1deg_ntl.csv")
metric="Gap_Filled_DNB_BRDF-Corrected_NTL"
ntl_df=ntl_df[(ntl_df["metric"]==metric)&(ntl_df["mean"]>=0)&
              (ntl_df["mean"]<65535)&(~ntl_df["mean"].isna())][["location","week","metric","mean"]]

**ColumnAmountNO2, ColumnAmountNO2Trop metrics are used from Nitrogen Dioxide**

In [ ]:
omino2_df=pd.read_csv(OMINO2_PATH+"indian_cities_1degx1deg_omino2.csv")
omino2_df["metric"].unique()

array(['ColumnAmountNO2', 'ColumnAmountNO2CloudScreened',
       'ColumnAmountNO2Trop', 'ColumnAmountNO2TropCloudScreened'],
      dtype=object)

In [ ]:
metrics=["ColumnAmountNO2","ColumnAmountNO2Trop"]
omino2_df=omino2_df[(omino2_df["metric"].isin(metrics))&(omino2_df["mean"]>-9999)&(~omino2_df["mean"].isna())]

**All metrics are needed to be converted to features (i.e. columns). So the final file will be for every location there should be every metric present for all weeks. All nan values are removed. For every row the values are present for every column/metric.**

In [ ]:
all_metrics=[]
data={}
for df in [airs_df,veg_df,sc_df,ntl_df,omino2_df]:
    locations=df["location"].unique().tolist()
    weeks=df["week"].unique().tolist()
    metrics=df["metric"].unique().tolist()
    all_metrics+=metrics
    print(metrics[0])
    for location in locations:
        if location not in data:
            data[location]={}
        location_df=df[df["location"]==location]
        for week in weeks:
            if week not in data[location]:
                data[location][week]={}
            metric_data=location_df[location_df["week"]==week].set_index("metric")["mean"].to_dict()
            data[location][week].update(metric_data)
                

SurfSkinTemp_TqJ_A
CMG 0.05 Deg 16 days NDVI
Eight_Day_CMG_Snow_Cover
Gap_Filled_DNB_BRDF-Corrected_NTL
ColumnAmountNO2


In [ ]:
data_list=[]
for location in data:
    for week in data[location]:
        vector=[location,week]+[data[location][week].get(metric,np.nan) for metric in all_metrics]
        data_list.append(vector)
df=pd.DataFrame(data_list,columns=["location","week"]+all_metrics)
df.dropna(inplace=True)

In [ ]:
df.to_csv(OUTPUT_PATH+"combined_1degx1deg.csv",index=False)